# Извлечение данных с помощью Gemma 1 2b

## Загрузка и установка Gemma из Kaggle

In [ ]:
!pip install -q -U tf-keras
!pip install -q -U keras-nlp==0.10.0
!pip install -q -U kagglehub>=0.2.4
!pip install -q -U keras>=3

In [ ]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

os.environ["KERAS_BACKEND"] = "jax"
#Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras_nlp
import keras
import json

## Извлечение данных из датасета с помощью Gemma

In [31]:
def extract_gas_basic_info(gemma_lm, sample):
    template = """Ты - ИИ-помощник, созданный для поиска информации о свойствах газа в тексте и её преобразовании в формат JSON.
Оформи ответ на вопрос в соответствии с примером. В ответе должно быть:
- название основного газа в составе;
- название газа;
- химическая формула газа;
- ГОСТ, задающий требования к качеству газа.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости: 1977
Код ООН: 1006
Физико-химические свойства:
Электронная конфигурация: 2s22p3
Молекулярая масса: 28.0134
Степень окисления: от +5 до -3
Плотность: 1,251
Удельная теплоёмкость: 1,034
Теплопроводность: 0,026
t кипения: -195,8 °C
t плавления: -210 °C
Внешние признаки: Без цвета вкуса и запаха, химически весьма инертен

ПРИМЕР ОТВЕТА: {"based_on":"на основе Азота","gas_name":"Азот жидкий особой чистоты 1 сорт","formula":"N","state_standard":"ГОСТ 9293 - 74"}
"""
    print("        Извлечение основной информации о газе...")

    prompt = template + "\nВОПРОС:\n" + sample + "\nОТВЕТ:"
    raw_response = gemma_lm.generate(prompt, max_length=1100)

    first_position = raw_response.find("{", raw_response.find("{") + 1)
    if first_position == -1:
        print("        ОШИБКА: Gemma не дала ответ. Возможная причина: слишком мало токенов.")
        print("        Завершено с ошибкой.")
        return None

    last_position = raw_response.find("}", first_position)
    if last_position == -1:
        print("    ОШИБКА: Gemma не сгенерировала корректную запись в формате JSON.")
        print("        Завершено с ошибкой.")
        return None

    try:
        result_json = json.loads(raw_response[first_position:last_position + 1])
        print("        Завершено.")
        return result_json
    except:
        print("        ОШИБКА: Не удалось преобразовать основную информацию о газе в JSON-структуру.")
        return None

In [32]:
def extract_gas_mark(gemma_lm, sample):
    template = """Оформи ответ в соответсвии с примером.

ПРИМЕР ВОПРОСА:
Гелий газообразный высокой чистоты, марка 4.6

ПРИМЕР ОТВЕТА:
{"mark":"4.6"}

ВОПРОС:
"""
    prompt = template + sample + "\n\nОТВЕТ:\n"

    print("        Извлечение марки газа...")
    raw_response = gemma_lm.generate(prompt, max_length=90)

    first_position = raw_response.find("{", raw_response.find("{") + 1)
    if first_position == -1:
        print("        ОШИБКА: Gemma не дала ответ. Возможная причина: слишком мало токенов.")

    last_position = raw_response.find("}", first_position)
    if last_position == -1:
        print("        ОШИБКА: Gemma не сгенерировала корректную запись в формате JSON.")

    try:
        result_json = json.loads(raw_response[first_position:last_position + 1])
        print("        Завершено.")
        return result_json
    except:
        print("        ОШИБКА: Не удалось преобразовать информацию о марке газа в JSON-структуру.")
        return None

In [33]:
def extract_gas_composition(gemma_lm, sample):
    template = """Ты - ИИ-помощник, созданный для поиска информации о составе газа в тексте.
Оформи ответ на вопрос в соответствии с примером. Каждый пункт ответа должен включать:
- название компонента газа из вопроса;
- химическую формулу компонента газа из вопроса;
- объёмную долю компонента газа из вопроса.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости: 1977
Код ООН: 1006
Физико-химические свойства:
Электронная конфигурация: 2s22p3
Молекулярая масса: 28.0134
Степень окисления: от +5 до -3
Плотность: 1,251
Удельная теплоёмкость: 1,034
Теплопроводность: 0,026
t кипения: -195,8 °C
t плавления: -210 °C
Внешние признаки: Без цвета вкуса и запаха, химически весьма инертен

ПРИМЕР ОТВЕТА:
* Азот - N - не менее 99.999%
* Кислород - O - не более 0.0005%
* Водород - H - не более 0.0002%

ВОПРОС:
"""
    #Периодически gemma начинает галлюцинировать. При некоторых вводах она упорно выводит
    #в ответе информацию про оксид азота, даже если в тексте о нём нет ни слова.
    #Проблемы также вызывает непоследовательность в формате таблиц.
    print("        Извлечение состава газа...")
    print("            Извлечение состава газа в виде списка...")

    prompt = template + sample + "\nОТВЕТ: "
    raw_response = gemma_lm.generate(prompt, max_length=1400)
    composition_list = ""
    for i in raw_response[raw_response.rfind("ОТВЕТ:")+6:len(raw_response)].split('*'):
        composition_list += i
    if len(composition_list) == 0:
        print("            ОШИБКА: Gemma не нашла данные о составе продукта.")
        print("        Завершено с ошибкой.")
        return None

    print("            Завершено.")
    print("            Преобразование списка в запись в формате JSON...")

    template = """Ты - ИИ-помощник, преобразующий списки в данные в формате JSON.
Оформи ответ на вопрос в соответствии с примером.

ПРИМЕР ВОПРОСА:
* Азот - N - не менее 99,999%
* Кислород - O - не более 0,0005%
* Водород - H - не более 0,0002%

ПРИМЕР ОТВЕТА: {"components":[{"name":"Азот","formula":"N","value":"99.999","operation":"не менее"},{"name":"Кислород","formula":"O","value":"0.0005","operation":"не более"},{"name":"Водород","formula":"H","value":"0.0002","operation":"не более"}]}

ВОПРОС:
"""
    prompt = template + composition_list + "\n\nОТВЕТ: "
    #При слишком большом окне модель начинает выводить ответ несколько раз подряд.
    response = gemma_lm.generate(prompt, max_length=1500)

    #Магическая последовательность из символов в ответе.
    index = response.find('{', response.find("ОТВЕТ:"))
    index2 = None
    if index == -1:
        print("            ОШИБКА: Gemma не дала ответ. Возможная причина: слишком мало токенов.")
        print("        Завершено с ошибкой.")
        return None
    else:
        if response[index] != '{':
            print("            ОШИБКА: Gemma не сгенерировала корректную запись в формате JSON.")
            print("        Завершено с ошибкой.")
            return None

        #Поиск конца записи в формате JSON и проверка скобок с помощью стека.
        stack = []
        for i in range(index, len(response)):
            if response[i] == '{':
                stack.append('{')
            elif response[i] == '}':
                if len(stack) == 0 or stack[-1] != '{':
                    break
                else:
                    stack.pop(-1)
            elif response[i] == '[':
                stack.append('[')
            elif response[i] == ']':
                if len(stack) == 0 or stack[-1] != '[':
                    break
                else:
                    stack.pop(-1)

            if len(stack) == 0:
                index2 = i
                break

        if index2 == None:
            print(response)
            print("            ОШИБКА: Gemma не сгенерировала корректную запись в формате JSON.")
            print("        Завершено с ошибкой.")
            return None

    try:
        result_json = json.loads(response[index:index2 + 1])
        print("        Завершено.")
        return result_json
    except:
        print("        ОШИБКА: Не удалось преобразовать информацию о составе газа в JSON-структуру.")
        return None

In [ ]:
def extract_info(gemma_lm, dataset):
    extracted_info = [[], [], []]
    extraction_error_count = [0, 0, 0]

    #файл 24 - хороший пример
    print("Извлечение информации из датасета...")
    for i in range(0,len(dataset)):
        print("    Обработка файла " + str(i) + "/" + str(len(dataset)) + "...")

        extracted_basic_info = extract_gas_basic_info(gemma_lm, dataset[i])
        extracted_info[0].append(extracted_basic_info)
        if extracted_basic_info == None:
            extraction_error_count[0] += 1
        else:
            if "gas_name" in extracted_basic_info:
                if extracted_basic_info["gas_name"].lower().find("марк") != -1:
                    extracted_mark = extract_gas_mark(gemma_lm, extracted_basic_info["gas_name"])
                    extracted_info[1].append(extracted_mark)
                    if extracted_mark == None:
                        extraction_error_count[1] += 1
                else:
                    extracted_mark = {"mark":"Отсутствует"}
                    extracted_info[1].append(extracted_mark)
            else:
                extracted_info[1].append(None)

        extracted_composition = extract_gas_composition(gemma_lm, dataset[i])
        extracted_info[2].append(extracted_composition)
        if extracted_composition == None:
            extraction_error_count[2] += 1

    print(("Завершено ("
           + str(extraction_error_count[0])
           + " ошибок извлечения основной информации, "
           + str(extraction_error_count[1])
           + " ошибок извлечения информации о марке газа, "
           + str(extraction_error_count[2])
           + " ошибок извлечения состава)."))

    print(extracted_info)
    return extracted_info

## Подготовка датасета, запуск Gemma, извлечение данных из датасета, сериализация и сохранение данных

In [59]:
import zipfile

def load_dataset(dataset_path):
    if not os.path.isdir(dataset_path):
        if not os.path.isfile(dataset_path):
            raise Exception("Error: file not found.")

        name, extension = os.path.splitext(dataset_path)
        print(extension)
        if extension == ".zip":
            with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
                zip_ref.extractall(name)
            dataset_path = name
        else:
            raise Exception("Error: unsupported file type.")

    dataset_file_names = os.listdir(dataset_path)

    dataset = []
    loaded_file_count = 0
    ignored_file_count = 0

    print("Загрузка датасета...")
    for i in dataset_file_names:
        f = open(dataset_path + "/" + i, 'r', encoding="utf-8")

        #Файлы, не содержащие информацию о ГОСТ или другом нормативном документе, игнорируются.
        buffer = f.readlines()
        ignore = True
        for j in buffer:
            if j.find("ГОСТ") != -1:
                ignore = False
                break
        if ignore:
            print("    ПРЕДУПРЕЖДЕНИЕ: Файл \"" + i + "\" не содержит информацию о ГОСТ. Файл пропущен.")
            ignored_file_count += 1
        else:
            loaded_file_count += 1
            buffer2 = ""
            for j in buffer:
                buffer2 += j
            dataset.append(buffer2)

        f.close()

    print("Завершено (" + str(len(dataset_file_names)) + " всего, " + str(loaded_file_count) + " загружено, " + str(ignored_file_count) + " пропущено).")
    return dataset

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

In [ ]:
import pickle

dataset = load_dataset("niikm_data.zip")
extracted_info = extract_info(gemma_lm, dataset)

print("Сериализация и сохранение извлечённой информации в \"gemma_extracted_info.bin\"...")
f = open("gemma_extracted_info.bin", "wb")
f.write(pickle.dumps(extracted_info))
f.close()
print("Завершено.")